In [1]:
%reload_ext XXX.mumble

In [2]:
    get_ipython().ast_node_interactivity='last_expr_or_assign'

    get_ipython().ast_node_interactivity='last_expr_or_assign'

In [3]:
    from poser import *

    from poser import *

In [11]:
    ---
    urls:
        prefix: http://results.enr.clarityelections.com/GA/
        reports: http://results.enr.clarityelections.com/GA/63991/184321/en/reports.html
        ga: http://results.enr.clarityelections.com/GA/
        elections: http://results.enr.clarityelections.com/GA/elections.json
        county: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        franklin: http://results.enr.clarityelections.com/GA/Franklin/64051/
        franklin_redir: http://results.enr.clarityelections.com/GA/Franklin/64051/183122/en/summary.html
        json: http://results.enr.clarityelections.com/GA/63991/184321/json/status.json
        select: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        wiki: https://en.wikipedia.org/wiki/List_of_counties_in_Georgia_(U.S._state)

    ---
    urls:
        prefix: http://results.enr.clarityelections.com/GA/
        reports: http://results.enr.clarityelections.com/GA/63991/184321/en/reports.html
        ga: http://results.enr.clarityelections.com/GA/
        elections: http://results.enr.clarityelections.com/GA/elections.json
        county: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        franklin: http://results.enr.clarityelections.com/GA/Franklin/64051/
        franklin_redir: http://results.enr.clarityelections.com/GA/Franklin/64051/183122/en/summary.html
        json: http://results.enr.clarityelections.com/GA/63991/184321/json/status.json
        select: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        wiki: https://en.wikipedia.org/wiki/List_of_counties_in_Georgia_(U.S._state)

In [12]:
    read_html, get = (a/cache)((a.pandas.read_html(), a.requests.get()))
    num_counties = (a[read_html].drop(2).first().len()[x-1])(urls['wiki'])

    read_html, get = (a/cache)((a.pandas.read_html(), a.requests.get()))
    num_counties = (a[read_html].drop(2).first().len()[x-1])(urls['wiki'])

159

In [14]:
# A List of All of the Counties

    soup = a[get][x.text].bs4.BeautifulSoup(features='lxml')(urls['county'])
    county = (a.map(x.attrs['value']).list())(soup.select('table tr li a'));
    assert len(county) is num_counties, """The number of rows is not the number of counties."""

# A List of All of the Counties

    soup = a[get][x.text].bs4.BeautifulSoup(features='lxml')(urls['county'])
    county = (a.map(x.attrs['value']).list())(soup.select('table tr li a'));
    assert len(county) is num_counties, """The number of rows is not the number of counties."""

In [17]:
# The County Event ID

    event_ids = a.map(a.str.split('/').filter(bool).take(2)).dict()(county);
    a.dict.items().take(3).dict()(event_ids)

# The County Event ID

    event_ids = a.map(a.str.split('/').filter(bool).take(2)).dict()(county);
    a.dict.items().take(3).dict()(event_ids)

{'Appling': '63993', 'Atkinson': '63994', 'Bacon': '63995'}

In [110]:
    def construct_context_version(name='Appling', sleep=1):
#         the.time.sleep(sleep)()
        latest_version = a[[a.identity(), event_ids.__getitem__]][
            [prefix], a.identity(), ['current_ver.txt']
        ].concat()['/'.join][get][x.text]

        constructed = '/'.join((prefix, name, event_ids[name], latest_version(name)))
        constructed = (constructed +'/json/{}.json').format
        return a.map(constructed).map((
            a.str.rsplit('/', 1).last().str.replace('.json', ''), 
            a[get].methodcaller('json'))).dict()(('sum', 'details', 'status'))

    def construct_context_version(name='Appling', sleep=1):
#         the.time.sleep(sleep)()
        latest_version = a[[a.identity(), event_ids.__getitem__]][
            [prefix], a.identity(), ['current_ver.txt']
        ].concat()['/'.join][get][x.text]

        constructed = '/'.join((prefix, name, event_ids[name], latest_version(name)))
        constructed = (constructed +'/json/{}.json').format
        return a.map(constructed).map((
            a.str.rsplit('/', 1).last().str.replace('.json', ''), 
            a[get].methodcaller('json'))).dict()(('sum', 'details', 'status'))

In [111]:
    ---
    labels:
        CH: Candidate
        C: Office
        V: Votes
        TV: Total Voters
        BC: Ballot Cast

    ---
    labels:
        CH: Candidate
        C: Office
        V: Votes
        TV: Total Voters
        BC: Ballot Cast

In [112]:
    def tidy_precinct(s):
        return a.zip().dict().valmap(
            a.zip(s.Candidate).dict()
        ).pandas.DataFrame()(s.P, s.V)

    def tidy_precinct(s):
        return a.zip().dict().valmap(
            a.zip(s.Candidate).dict()
        ).pandas.DataFrame()(s.P, s.V)

In [116]:
    dfs = {}
    for i, county in enumerate(event_ids):
        print(i)
        objects = construct_context_version(county)
        
        df = (a.pandas.DataFrame())(objects['sum']['Contests'])
        
#         assert df.V.apply(sum).lt(df.TV).all()
        df = df[list(labels)].rename(columns=labels)
        df = (a.pandas.DataFrame()[df.join])(objects['details']['Contests'])

        dfs[county] = a.pandas.concat()({index: tidy_precinct(row)
            for index, row in df.set_index('Office').iterrows() 
            if (a**list)(row.P) or not a.pandas.np.isnan()(row.P)})

    dfs = {}
    for i, county in enumerate(event_ids):
        print(i)
        objects = construct_context_version(county)
        
        df = (a.pandas.DataFrame())(objects['sum']['Contests'])
        
#         assert df.V.apply(sum).lt(df.TV).all()
        df = df[list(labels)].rename(columns=labels)
        df = (a.pandas.DataFrame()[df.join])(objects['details']['Contests'])

        dfs[county] = a.pandas.concat()({index: tidy_precinct(row)
            for index, row in df.set_index('Office').iterrows() 
            if (a**list)(row.P) or not a.pandas.np.isnan()(row.P)})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158


In [139]:
    tidy = a.pandas.concat()(dfs)

    tidy = tidy.swaplevel(0,1).stack().pipe(
        lambda df: a.pandas.Series()(df.values, df.index.rename(('contest', 'county', 'candidate', 'precinct')))
    ).rename('votes')

    tidy = a.pandas.concat()(dfs)

    tidy = tidy.swaplevel(0,1).stack().pipe(
        lambda df: a.pandas.Series()(df.values, df.index.rename(('contest', 'county', 'candidate', 'precinct')))
    ).rename('votes')

contest                         county   candidate           precinct    
CC, D2                          Appling  T. WILKERSON (I) D  2               945.0
CC, D4                          Appling  R. RENTZ (I) R      4A              248.0
                                                             4B              274.0
                                                             4C              255.0
                                                             4D              625.0
CC, D5                          Appling  C. LEGGETT (I) R    5A              513.0
                                                             5B              327.0
                                                             5C              225.0
CLERK SUPERIOR COURT            Appling  M. THOMAS (I) R     1B              692.0
                                                             1C              481.0
                                                             2               801.0
             

In [148]:
    tidy.to_csv('all_contests_precinct.csv')
    tidy.loc['President of the United States'].to_csv('president_precinct.csv')

    tidy.to_csv('all_contests_precinct.csv')

    tidy.loc['President of the United States'].to_csv('president_precinct.csv')

> I want election **results** by **precinct** in **Georgia** for the **2016 presidential election**. The problem is that results are broken down by county in details.xls on the Georgia Secretary of State's website. However, precinct-level details are available for each of Georgia's 159 counties 

![](./state_list.png)

in individual reports (also titled details.xls). 

![](./county_example.png)